# Pandas

In [ ]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
help(pd)

Like the data.frame in R, the functionality offered by pandas revolves around in-memory data manipulation.<br><br>
There are two important classes defined in the pandas library:
1. **Series**, a one-dimensional ndarray with *labeled axis* (including time series) & 
2. **DataFrame**, a two-dimensional size-mutable, potentially heterogeneous tabular data structure with *labeled axes*

DataFrames can be thought of as a dict-like container for Series objects. The *labeled axes* a.k.a. indices is what makes pandas fast and powerfull, but also sometimes confusing if you come from you are used to other datasets like R's data.frame. Give yourself time to get used to the concepts, it will  be worth it.

# Series

A series (think time-series) is a sequence of values where the elements are labelled **with an index**.<br>
You can think of a Series a generalized 1D numpy array.<br>
When no index is explicitly specified, pandas uses by default the sequence 0 ... n.<br>
When working with timeseries the index is usually a timestamp.<br>

In [ ]:
## the easiest to create a Series is probably
s1 = pd.Series([2,5,3,4], name='My First Series')
s1
## note the column on the left (0, ..., 3) is the index column

## Main Components

In [ ]:
## The main pandas objects consist of three major parts
print('Name         :', s1.name)
print('Values       :', type(s1.values), 'with values', s1.values)
print('Index/Labels :', s1.index)
print('Data Type    :', s1.dtype)

A Series can be viewed as a **generalized numpy array**.<br>
As shown above the actual data in a series lives in a numpy.ndarray.<br>
The difference is that pandas Series (& DataFrames) have row **labels contained as an index**.<br>
An index can be seen as an immutable array.<br>
Another way of thinking about a Series is as a specialized dictionary, where the index values are the keys mapping to the Series values.

# Data Types

Before we dive into the ways pandas can be used to manipulate data, I want to touch on the types of data that can live in a pandas Series / DataFrame.
<br><br>
Pandas is build on top of NumPy. Therefore **pandas offers all the datatypes NumPy offers**.<br>
A numpy array consists of a contigeous linear collection of elements. When the elements are integer, or float, or some other type that uses a given number of bits in memory, pandas is fast and efficient. The numpy type system is very rich, see the [documentation](https://numpy.org/devdocs/user/basics.types.html?highlight=data%20types) for much more info. As a rough summary:

| Class | Type | Remarks | Character Code |
|---|---|---|---|
| **Booleans:** | bool_| compatible: Python bool | '?' |
| &nbsp; | bool8 | 8 bits | |
| **Integers:** | byte | compatible: C char | 'b' |
| &nbsp; | short | compatible: C short | 'h' |
| &nbsp; | intc | compatible: C int | 'i' |
| &nbsp; | int_ | compatible: Python int | 'l' |
| &nbsp; | longlong | compatible: C long long | 'q' |
| &nbsp; | intp | large enough to fit a pointer | 'p' |
| &nbsp; | int8 | 8 bits | 'int8' |
| &nbsp; | int16 | 16 bits | 'int16' |
| &nbsp; | int32 | 32 bits | 'int32' |
| &nbsp; | int64 | 64 bits | 'int64' |
| **Unsigned integers:** | ubyte | compatible: C unsigned char | 'B' |
| &nbsp; | ushort | compatible: C unsigned short | 'H' |
| &nbsp; | uintc | compatible: C unsigned int | 'I' |
| &nbsp; | uint | compatible: Python int | 'L' |
| &nbsp; | ulonglong | compatible: C long long | 'Q' |
| &nbsp; | uintp | large enough to fit a pointer | 'P' |
| &nbsp; | uint8 | 8 bits | 'uint8' |
| &nbsp; | uint16 | 16 bits | 'uint16' |
| &nbsp; | uint32 | 32 bits | 'uint32' |
| &nbsp; | uint64 | 64 bits | 'uint64' |
| **Floating-point numbers:** | half | &nbsp; | 'e' |
| &nbsp; | single | compatible: C float | 'f' |
| &nbsp; | double | compatible: C double | |
| &nbsp; | float_ | compatible: Python float | 'd'
| &nbsp; | longfloat | compatible: C long float | 'g'
| &nbsp; | float16 | 16 bits | 'f2' |
| &nbsp; | float32 | 32 bits | 'f4' |
| &nbsp; | float64 | 64 bits | 'f8' |
| &nbsp; | float96 | 96 bits, platform? | |
| &nbsp; | float128 | 128 bits, platform? | |
| **Complex floating-point numbers:** | csingle | &nbsp; | 'F' |
| &nbsp; | complex_ | compatible: Python complex | 'D' |
| &nbsp; | clongfloat | &nbsp; | 'G' |
| &nbsp; | complex64 | two 32-bit floats | &nbsp; |
| &nbsp; | complex128 | two 64-bit floats | &nbsp; |
| &nbsp; | complex192 | two 96-bit floats, platform? | &nbsp; |
| &nbsp; | complex256 | two 128-bit floats, platform? | &nbsp; |
| **Any Python object:** | object_ | any Python object | 'O' |


As in any other language, when the elements are of variable length, what gets captured is the reference. The actual memory used is allocated outside the array.<br>
Take strings, strings typically have variable lengths. So when we have a column of strings pandas stores an array of string object references.<br>
Each reference points to a structure in memory that holds some properties (like: length) and the phisical location of a buffer where the len characters of the string are stored:

In [ ]:
## don't worry to much if you do not understand all syntax here
answcat = ['strongly disagree', 'disagree', 'neutral', 'agree', 'strongly agree']
s1 = pd.Series(np.random.choice(answcat,p=[0.2]*5,size=50))
s1.head()

In [ ]:
type(s1[0])

We can use the string via the reference just like any other string in Python:

In [ ]:
s1[0].upper()

But, note that the dtype = object. This is pandas way of telling you that it is keeping object references.<br>
To explain this a bit further, lets look at the memory_uasege():

In [ ]:
s1.memory_usage()

This memory usage describes what the actual numpy arrays uses, but does not count the actual memory taken by the strings. To get that do: 

In [ ]:
s1.memory_usage(deep=True)

Keep from this, that pandas can store numeric data very efficient using numpy arrays. All other types get stored using an object reference.<br>
Working with these object reference is considerably slower, the reference needs to be followed and interpreted each time an object reference is used.<br>

## Categorical Data

An important way to speed up pandas is to use categorical wherever applicable.<br>
A categorical variable is stores a code that serves as a key into a lookup table where the associated strings / labels are kept.<br>
Under the hood pandas works with these codes, which is considerable faster than working with object references to strings.<br>
As an added benifit, it is much more efficient way to store the data!

In [ ]:
anscat_categorical = {1:'strongly disagree', 2:'disagree', 3:'neutral', 4:'agree', 5:'strongly agree'}
example_short = np.random.choice(range(1,6),p=[0.2]*5,size=50)
example_long  = np.array([anscat_categorical[e] for e in example_short])
df1 = pd.DataFrame({'short':example_short, 'long': example_long})
print(df1.memory_usage(deep=True))
df1.head()

A way to make the variable *categorical* in pandas is using the astype('categorical'):

In [ ]:
df2 = df1.copy()
df2.long = df2.long.astype('category')
print(df2.memory_usage(deep=True))
df2.head()

In [ ]:
print(f'df1 dtype: {df1.long.dtype}\ndf2 dtype: {df2.long.dtype}')

As you can see, the dtype is now **category** with possible values listed.<br>

In [ ]:
df2.long.dtype

In [ ]:
df2.long.memory_usage(deep=True) / df1.long.memory_usage(deep=True)

We are using less than 20% of the memory, in real world examples this will often be much more dramatic (if the number of possible strings is small and the number of rows is large).<br>
**Just as important, all sort of other data manipulations become much faster (grouping, aggregations, selections, ...)**

# Index

## RangeIndex

Pandas automatically creates an index if none is given. The index created will be a RangeIndex, representing a sequence of integers.

In [ ]:
s1 = pd.Series([7,5,3], name='My Series')
print(s1,'\n')
print(s1.index)

Apart from the default index that gets created if you do not supply one, you can create one explicitely.

In [ ]:
## we also get a RangeIndex when we specify: index=range(...)
s1 = pd.Series([7,5,3], name='My Series', index=range(5,8))
print(s1,'\n')
print(s1.index)

The index values relates to a specific element, and does not change if an element is removed:

In [ ]:
## drop returns a new Series with the specific index value removed, but leave the original unchanged, unless ...
## the argument inplace=True is given.
s1.drop(6, inplace=True)
s1

The remaining elements keep their index value.<br>
Also note that RangeIndex(start=5, stop=8, step=1) can no longer be valid.<br>
Under the hood, pandas managed this by changing the index type:

In [ ]:
s1.index

There is a whole hierarchy of Index classes defined in pandas.<br> 
For daily use, you let pandas take care of the details.

In [ ]:
## for the interested, or when you need it, you can find out much about the hierarchy using the %pserach magic
%psearch pd.core.indexes.[a-zA-Z]*

## Index: Any Hashable Type

An index doesn't have to be integer based. It can be, and often is, strings:

In [ ]:
s1 = pd.Series([7,5,3], name='My Series', index=['Jan','Feb','Mar'])
print(s1,'\n')
print(type(s1.index), s1.index)

Actually, an **index can be any hashable data type**.<br><br>
A hashable data type is a data type that provides the method \_\_hash\_\_(). Roughly speaking all primitive data types are hashable and the unmutable collections are hashable as well. As examples, lists and dicts are not allowed, but you could use a tuple as an index.

In [ ]:
## many types are hashable in Python: like strings
'a string'.__hash__()

In [ ]:
## or, more adventurous, functions
def greater_than_two(a): return(a>2)
greater_than_two.__hash__()

In [ ]:
## or tuples
(1,2,3).__hash__()
## please note that tuples are also used in MultiIndex (discussed below), so the syntax has some gotcha's

In [ ]:
## as an example: to create a Series with an index of type tuple 
s1 = pd.Series([11,22,33], name='s1', index=[(1,1),(2,2),(3,3)])
print(s1,'\n')
print(type(s1.index))

Also, an index does not have to be unique:

In [ ]:
## note: list('aba') --> ['a','b','a']
s3 = pd.Series([1,2,3], name='s1', index=list('aba'))
s3

## DateTimeIndex

An important type of index used in **time series** is the **DatetimeIndex**.<br>
The DatetimeIndex class provides lots of functionality around date time based indices.<br><br>
Pandas provides some usefull functions to generate these, like **date_range**:

In [ ]:
## to create an index of a sequence of consecutive days, use freq='D'
pd.date_range(start='2019-03-26', end='2019-04-02', periods=7) ## freq='D')

So if we have data containing daily maximum temperatures, a natural way to store this is given by:

In [ ]:
idx = pd.date_range(start='2019-03-26', end='2019-04-02', freq='D')
daily_max_temp_degree_c = pd.Series([12,13,14,13,15,14,15,15], name='Daily Max Temp C', index=idx)
daily_max_temp_degree_c.plot()

It is also possible to specify multiples in the *freq=* notation:

In [ ]:
## so every Monday from 2019-03-25 to 2019-04-05 is - the day_name() method returns the day name ...
print('Check: 2019-03-25 is a', pd.to_datetime('2019-03-25').day_name())
## to define multiples, put an integer before the period, so: freq='D' is daily and freq='7D' is weekly
pd.date_range(start='2019-03-25', end='2019-04-17', freq='7D')
## note that '2019-04-15' is included

In [ ]:
## or let's say we want two Mondays
pd.date_range(start='2019-03-25', periods=2, freq='7D')

In [ ]:
## or something with working days using freq='C'
idx = pd.date_range(start='2019-03-26', end='2019-04-02', freq='C')
pd.Series([f"{dt.day_name()} work {hr}Hr {':-)' if (hr<8) else ':-('}" for dt,hr in zip(idx, [8,8,8,3,8,8])], 
          name='Work Pattern', 
          index=idx)

[Link](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases) to more info on frequency aliases.

Pandas provides usefull methods to down/upsample timeseries using **resample**:

In [ ]:
## create a slightly bigger Series, to demonstrate down sampling
s4  = pd.Series(np.random.randn(35), index=pd.date_range(start='2019-03-26', periods=35, freq='D'))
s4.head(n=7)

In [ ]:
## to down sample to one record per week --> taking the last value of the week
s4.resample('W', label='right').last()
## label='right' means: the interval is labelled using the 'ending Sunday'
## .last() means: take the last value in the interval -- other option: first, mean, median, ...

In [ ]:
## check what pandas is doing ... selecting the Sunday's --> see discussion on selection later :-)
s4.loc[s4.index.day_name() == 'Sunday']

In [ ]:
## more generically, instead of using one of the many predefined methods, like: last, first, mean, std, ...
## we could define our own aggregation function, for instance: lag 1 autocorrelation:
def my_aggregation_fun(srs): return(np.NaN if (len(srs)<4) else srs.autocorr(lag=1)) ## NaN is len < 4
s4.resample('M', label='right').agg(my_aggregation_fun)

*.resample()*:
1. *splits* the series into a set of smaller series (as defined by the first argument),
2. *apply* the function, here my_aggregation_fun, and 
3. *combines* the result back into a resulting Series.

This **split-apply-combine** pattern is present everywhere in data-science workflows, and pandas has some nice syntax to do this using *groupby()* on which more later.

## Index Viewed As Set

In many ways an index behaves like a set:

In [ ]:
s51 = pd.Series([7,5,3,7,1], index=pd.Index([1,2,3,4,5]))
s52 = pd.Series([4,6,3,2,8], index=pd.Index([    3,4,5,6,7]))

In [ ]:
## when using the in operator, pandas checks if the element in the index
print(f'check: 7 in s51 gives: {7 in s51}')
print(f'check: 2 in s51 gives: {2 in s51}')
## equivalent to: _ in s51.index

More, specifically, the operators: &, |, and ^ behave like set operations

In [ ]:
## the indices that are in both sets --> logical AND
s51.index & s52.index

In [ ]:
## the indices that are in either sets --> logical OR
s51.index | s52.index

In [ ]:
## the indices that are in only one of the sets --> logical XOR
s51.index ^ s52.index

**GOTCHA**: an index behaves like an *immutable array* for the math operators: -, +, \*, and /

In [ ]:
print(s51.index + s52.index)
print(s51.index - s52.index)
print(s51.index * s52.index)
print(s51.index / s52.index)

## MultiIndex

Finally, an index can have multiple levels.<br>
As an example assume we have end of month data on accounts.<br>
Using the from_arrays() method on the MultiIndex class:<br>

In [ ]:
s6  = pd.Series(np.random.randn(9), 
                name    = 'Fictitious Balance', 
                index   = pd.MultiIndex.from_arrays([  [1,1,1,2,2,2,3,3,3], 
                                                       ['201801','201802','201803']*3
                                                    ], 
                                                    names=['account','yyyymm']
                                                   )
              )
s6

In [ ]:
s6.index

This becomes rather usefull when selecting or aggregating:

In [ ]:
s6.loc[:,'201803']

Many aggregation methods defined on the Series (and DataFrames) take *level=* as argument:

In [ ]:
## aggregate by account
s6.sum(level='account')

In [ ]:
## aggregate by year/month
s6.sum(level='yyyymm')

There will be more on aggregating, slicing, & dicing data later on. For now, observe that MultiIndex (or hierarchical indexing) can lead to natural / readable syntax.

# DataFrame

The **DataFrame** lies at the hart of many Data Science workflows. It holds a two-dimensional table of data where each column can be be of a different data type. A DataFrame is build from a set of Series, all sharing the same index.<br><br>
Let's start by creating a DataFrame from two Series:

In [ ]:
## lets create some random data
s1 = pd.Series(np.random.randn(4), name='series1', index=[1,2,3,4])
s2 = pd.Series(np.random.randn(4), name='series2', index=[3,4,5,6])
s3 = pd.Series(np.random.randn(4), name='series3', index=[5,6,7,8])
## the __init__() method of DataFrame accepts a dict of Series
df = pd.DataFrame({'s1': s1, 's2': s2, 's3': s3})
df

Note that pandas automatically lines up the indices and fills with NaN where the values are missing!

## Main Components

Just like with Series, the three important part of a DataFrame are:
* columns ==> column index
* index   ==> row index
* values  ==> the data

In [ ]:
df.columns

In [ ]:
df.index

In [ ]:
df.values

To get some summary information regarding the DataFrame, use *.info()*:

In [ ]:
df.info()

# Selecting Data

We already used indexing / slicing / selecting data from a Series.<br>
In the following we will take a deep-dive into selecting data from a DataFrame.<br><br>

There are two ways to reference elements of the series:
1. **.loc[]** & **.at[]**: using **index values** & 
2. **.iloc[]** & **.iat[]**: **location based** (n-th element & not element with index n)

Note: it is also possible to use the **[]** operator directly.<br>
But, it can be confusing. To be discussed last, once the basics are nailed down
<br><br>
Direct use of operator[] is usefull in the Notebook, but discouraged in production code.<br>
(from the docs) ... *since the type of the data to be accessed isn’t known in advance, directly using standard operators has some optimization limits. For production code, we recommended that you take advantage of the optimized pandas data access methods exposed in this chapter*.

## Single Element

Let's look at some simple examples: *.at[]* & *.iat[]*

In [ ]:
df

In [ ]:
## element with row index = 1 and column (index) = s1 (position 0, 0)
df.at[1,'s1']

In [ ]:
## the same element now using it's position 0, 0
df.iat[0,0]

This is basically all you can do with *.at[]* and *.iat[]*.<br>
They are simple and fast, but less flexible as *.loc[]* and *.iloc[]*.<br>
You could do the same thing using *.loc[]* and *.iloc[]*:

In [ ]:
df.loc[1,'s1']

In [ ]:
df.iloc[0,0]

But *.loc[]* and *.iloc[]* can also deal with more complicated selections.
<br><br>
The use case for *.at[]* & *.iat[]* is simply speed ...
<br><br>
*From the documentation*: Since indexing with **.loc[]** & **.iloc[]** must handle a lot of cases (single-label access, slicing, boolean indexing, etc.), it has a bit of overhead in order to figure out what you’re asking for. If you only want to access a scalar value, the fastest way is to use the **at[]** and **iat[]** methods.

### Timeit

In [ ]:
rs = pd.Series(0,range(10000))
df = pd.DataFrame(np.random.randn(10000,2))

In [ ]:
%%timeit
for i in rs.index: rs.loc[i] = df.loc[i,0] + df.loc[i,1]

In [ ]:
%%timeit
for i in range(10000): rs.iloc[i] = df.iloc[i,0] + df.iloc[i,1]

In [ ]:
%%timeit
for i in rs.index: rs.at[i] = df.at[i,0] + df.at[i,1]

In [ ]:
%%timeit
for i in range(10000): rs.iat[i] = df.iat[i,0] + df.iat[i,1]

The *.at[]* and *.iat[]* show some pretty good speedups here ...
<br><br>
But ... you should **always use 'vectorized computation'** in pandas and numpy (and R and probably any intepreted language that does not have compiler optimizations to make it fast), as show below:

In [ ]:
%%timeit
rs = df.sum(axis=1)

Below we delve into more complex selection mechanisms that allow for vecorized computations on mor complex sub-selections.

## Boolean Indexing

A pattern often used is boolean indexing. To quickly select a subset where some condition holds, we pass in a Series (or array or list) of boolean values.<br>
In pandas boolean indexing is supported using *.loc[]*:

In [ ]:
df = pd.DataFrame(np.random.randn(10,3), columns=['c1','c2','c3'])
df

In [ ]:
## define a simple expression that returns a Series of booleans where column c1 > 0
df.c1 > 0

In [ ]:
## to select all the rows where this condition is True
df.loc[df.c1 > 0]

If you want to use multiple conditions, you **must wrap each condition in ()**:

In [ ]:
df.loc[(df.c1 > 0) & (df.c3 < 0)]

Another way of doing the same is using the query method:

In [ ]:
df.query('c1 > 0 & c3 < 0')

This way of working with your data is what makes pandas work. If you work with poandas (or numpy, R data frames, or ...) you need to become familiar at this way of working. Working with loops will slow your code down and make it much less readable and maintainable.
<br><br>
Let's replace all the negative elements with 0, the bad way and the good way:

In [ ]:
df = pd.DataFrame(np.random.randn(10000,2), columns=['c1','c2'])
df.head()

In [ ]:
%%timeit
## The BAD way. Don't do this!!!!
res = df.copy()
for row_index in range(res.shape[0]):
    for col_index in range(res.shape[1]):
        if res.iat[row_index,col_index] < 0:
            res.iat[row_index,col_index] = 0

In [ ]:
%%timeit
## A better way ...
res = df.copy()
## loop over columns (more on iterating over Series and DataFrames later)
for _, col in res.items():
    ## select elements where col < 0 is True and replace with 0
    col.loc[ col < 0 ] = 0

In [ ]:
%%timeit
## The Good way. Instead do this!!!!
res = df.copy()
## shorter, prettier, and arguably more readable ...
res.where( res < 0, 0 )

## Using List or Array

In [ ]:
df = pd.DataFrame(np.random.randn(10,3), columns=['c1','c2','c3'])
df

In [ ]:
df.loc[[1,6,7,9]]

In [ ]:
df.loc[[1,6,7,9],['c3','c1']]

In [ ]:
## note that negative index values are allowed
df.iloc[[3,2,1], ::-1]

## Slicing

Pandas can select data using the slice notation *lower*:*upper*:*step*<br><br>
**GOTCHA**: index-based slicing using .loc[] **includes** the upper bound!!

In [ ]:
df = pd.DataFrame(np.random.randint(low=1, high=9, size=(5,5)), columns=['c1','c2','c3','c4','c5'])
df

In [ ]:
## rows with index values 1,2, & 3 (upper bound included in index slicing)
df.loc[1:3]

In [ ]:
## rows at position 1 & 2 (upper bound not included in position slicing)
df.iloc[1:3]

In [ ]:
df.loc[1:3,'c1':'c4']

In [ ]:
## first two rows by first two columns
df.iloc[:2,:2]

Since the index does not have to consist of unique values and the index does not have to be increasing or decreasing, slicing with **index** can fail or give unexpected results.<br>
The process is to obtain a index slice is:
1. get unique position of left value (throws KeyError if not unique)
2. get unique position of right value (throws KeyError if not unique)
3. return the sequence between these positions

In [ ]:
## lets create a series with an index that is not strictly increasing or decreasing and contains duplicate values 
df = pd.Series([1,2,3,4,5,6,7,8],index=list('acdfebae'))
## index value 'a' not unique
try:
    print(df.loc['a':'d'])
except Exception as e:
    print(e)
## index value 'e' not unique
try:
    print(df.loc['b':'e'])
except Exception as e:
    print(e)
## index values 'f' & 'b' unique, but ... does this make sense?
try:
    print(df.loc['f':'b'])
except Exception as e:
    print(e)

In [ ]:
df.index.is_unique

In [ ]:
df.index.is_monotonic

Slicing using the **position** based syntax is always unambiguous!

## Callable Function

Finally, you can pass in a function (or callable) that takes one argument (the calling Series or DataFrame) and returns valid output for indexing.

In [ ]:
np.random.seed(1)
df = pd.DataFrame(np.random.randint(low=1, high=9, size=(5,5)), columns=['c1','c2','c3','c4','c5'])
df

In [ ]:
## all rows that have a column max
def select_rows_with_1s(df): 
    return((df==1).sum(axis=1) > 0)

## all rows that have a column containing a column max
def select_rows_with_column_max(df):
    return(set(df.idxmax(axis=0)))

## put the functions in a dict
funs = {'1': select_rows_with_1s, '2': select_rows_with_column_max}

In [ ]:
ixfun = input('''
1: to select all rows that have at least one 1
2: to select all rows that contain a column max
Choose your function: ''')

df.loc[ funs[ixfun] ]

## Shortcut: [] directly on DataFrame[]

Finally, you can use the syntax *DataFrame[]*. This syntax is **very** confusing if you do not understand the basics. The behaviour of *DataFrame[]* depends on the arguments given:

In [ ]:
## lets create a dataframe that shows of the behaviour.
## spend some time in understanding this, it will pay off!!
df = pd.DataFrame(np.random.randn(5,3), index=[1,2,3,4,5], columns=[1,2,3])
df

Single value --> argument interpreted as column index value / name

In [ ]:
df[1]

List of values --> argument interpreted as list of column index values / column names

In [ ]:
df[[3,1]]

List of booleans --> argument interpreted as list of booleans for the rows

In [ ]:
df[[True,False,False,True,False]]

Slice --> argument interpreted as a row slice using position (0-based)

In [ ]:
## position 1 is the second row & since we are doing positional slicing the upper bound 4 is excluded
df[1:4]
## note: 1,2,3 (excluding 4) in position have index values: 2,3,4

You often see these selections chained:

In [ ]:
df[[True,False,False,True,False]][[3,1]]

In summary:

| DataFrame[] | DataFrame.loc[] / DataFrame.iloc[] | position / index |
| --- | --- | --- |
| df[1] | df.loc[:,1] | index based |
| df[[1,2]] | df.loc[:,[1,2]] | index based |
| df[[True, False, ...]] | df.loc[[True, False, ...],:] | n/a |
| df[1:4] | df.iloc[1:4,:] | position based |

Two final remarks:
1. when column names do not clash with the Python reserved words / special characters then<br>
```df['col1']``` is equivalent to ```df.col1``` (**this get used all the time!**)
2. use: ```df[condition] = replacement``` to replace all values in a DataFrame base on a matrix of booleans

In [ ]:
df.rename(columns={1:'col1',2:'col2',3:'col3'}, inplace=True)
df

In [ ]:
df.col2

In [ ]:
df['col2']

In [ ]:
df<0

In [ ]:
df[df<0] = 0
df

## Non-Existing Row/Column Index

Referencing a key that does not exist, will throw a KeyError exception.<br>
Assigning to a key that does not exist, will add the index and assign the value!

In [ ]:
df = pd.DataFrame(42,index=['a','b'],columns=['c1','c2'])
df.loc['c']

In [ ]:
df

In [ ]:
df.loc['c','c2'] = 42
df

In [ ]:
df.loc['c','c3'] = 42
df

In [ ]:
df.loc[:,'c4'] = 84
df

In [ ]:
df['c5'] = df.sum(axis=1)

In [ ]:
df['c6'] = 33
df

Note that since a DataFrame is a collection of Series, indexed by their name (aka column name):
* adding a column using the above syntax is efficient
* adding a row by using the above syntax is very inefficient. Worse case:
  * a new numpy.array for each column needs to be made
  * all the data needs to be copied over
  * the original deleted (by the garbage collector at some later point)

## IndexSlice For Complex MultiIndex Cases

In certain complex multi-index cases slicing is not possible using tuple notation.<br>
More specifically when you want to keep one level of a multi index fixed and slice other levels.<br>
This sound complicated, so lets generate an example:

In [ ]:
df = pd.DataFrame( np.random.randn(15,9), 
                   index   = pd.MultiIndex.from_arrays([  [1]*5 + [2]*5 + [3]*5, 
                                                          ['201801','201802','201803','201804','201805']*3
                                                       ], 
                                                       names=['account','yyyymm']
                                                      ),
                   columns = pd.MultiIndex.from_arrays([  ['atm']*3 + ['contactless']*3 + ['mobile']*3,
                                                          ['nr','vol','avg']*3
                                                       ], 
                                                       names=['account','yyyymm']
                                                      )
                 )
df

In [ ]:
df.index.levels

In [ ]:
df.columns.levels

Lets assume we want the atm data for accounts 1 & 2:

In [ ]:
df.loc[1:2,'atm']

So selections on levels[0] are straightforward, the problem occurs when we want to incorporate different levels. The *.loc[]* can deal with one or two parameters, not more ...
<br><br>
Lets assume we want the 'avg' data for yyyymm: '201801', '201802', & '201803'<br>
What we want is using the tuple natation like:<br>
```
df.loc[(:,['201801','201802','201803']),(:,'avg')]
```

In [ ]:
df.loc[(:,'201801':'201803']),(:,'avg')]

The slice operator : does not work inside the tuple brackets.<br>
This is where the IndexSlice class comes in, we can pass IndexSlice objects into *.loc[]*, where each IndexSlice can be a complex definition of rows / columns to select:

In [ ]:
df.loc[ pd.IndexSlice[:, '201801':'201803'],
        pd.IndexSlice[:, 'avg']
      ]

# Copy Or View

Most methods in pandas will return a copy of the data, but not always!<br>
This is a potential minefield in pandas<br><br>

Here's the rules:
1. If **inplace=True** is provided, it will **modify in-place** (only some operations)
2. An indexer that **sets** using (.loc[]/.iloc[]/.at[]/.iat[] = ) will **set inplace**.
3. An indexer that gets **(part of a) Series** / **complete DataFrame** is **almost always a view**.
4. An indexer that gets a **more complicated subset** of the original is always a **copy**.
5. All **operations** / **methods** generate a **copy**.

In [ ]:
## generate some toy data
s1 = pd.Series(np.random.randn(3), name='s1', index=[0,1,2])
s2 = pd.Series(['a','b','c'],      name='s2', index=[1,2,3])
df = pd.DataFrame({'s1': s1, 's2': s2})
df

In [ ]:
## rule 3: get part of single Series --> view
## change the original & the change is reflected in the view
tmp = df.loc[:3,'s1']
df.loc[1,'s1'] = 99
tmp

In [ ]:
## rule 3: complete DataFrame --> view
## change the original & the change is reflected in the view
tmp = df
df.loc[2,'s2'] = 'X'
tmp

In [ ]:
## rule 4: subset of rows and columns --> copy
## change the original & the change is NOT reflected in the copy
tmp = df.loc[:3,['s1','s2']]
df.loc[2,'s1'] = 66
tmp

One question that pops up often on StackOverFlow is about the dreaded SettingWithCopyWarning:

In [ ]:
df[3:4]['s1']
## we will get a copy ... if we subsequently update the value --> SettingWithCopyWarning
## to enable setting the value of an indexer, use .loc[]/.iloc[]/.at[]/.iat[] --> these will set inplace

In [ ]:
df[3:4]['s1'] = 33
df

In [ ]:
df.loc[3:4,'s1'] = 33
df

Since df[3:4]['s1'] returns a copy, and the pandas can deduce that an element is being set ... it warns you that you are setting an element on a temporary copy ... probably not what you were expecting. This error occurs regularly when you are chaining multiple selections.<br><br>
Another form of this error is when you create a varaiable from a view, and later change a value. It's pandas way of warning you: you're about to change the value of the original by setting a value on the view ...<br>
Note that the error can potentially pop up some distance away from where a copy was taken ...

In [ ]:
tmp = df[3:4]['s2']
## more code here
## ...
## more code here
tmp.iloc[0] = 'X'
df

In [ ]:
## below is fine, because the copy() is explicitly called
tmp = df[3:4]['s2'].copy()
## more code here
## ...
## more code here
tmp.iloc[0] = 'X'
df

# Operators

In object oriented programming languages the behaviour of operators like: +, -, /, \*, are often implemented to give them intuitive usage.<br>
It's what makes NumPy and SciPy (or Matlab, or ...) so elegant for computation.<br><br>
Just like in NumPy, where a calling an operation on an array returns an array, in pandas many operations are defined for Series and DataFrames and return Series or DataFrames.

## Logical

We have allready used these throughout ...

In [ ]:
s1 = pd.Series([-1,3,-2,5,8])

In [ ]:
s1 > 0

In [ ]:
df = pd.DataFrame([[-1,3],[-2,5],[8,-2]])
df

In [ ]:
## you can do things like
df[df<0] = 0
df

## Mathematical

In [ ]:
(s1 + 5) // 3

Since Series and DataFrames consist of NumPy arrays, we can do things like:

In [ ]:
(s1 - np.mean(s1)) / np.std(s1)

In [ ]:
type((s1 - np.mean(s1)) / np.std(s1))

Or lets say we want to normalize a column in a DataFrame to [0,1]:

In [ ]:
df = pd.DataFrame(np.random.randint(1,51,size=(5,2)), columns=['r1','r2'])
df

In [ ]:
## note that we can add a column by assignment, see section 5.7
df['r1_norm'] = (df.r1 - np.min(df.r1)) / (np.max(df.r1) - np.min(df.r1))
df

# Methods

Many methods defined for numpy arrays are also implemented for Series.<br>
For instance, instead of using the Numpy min & max we could do:

In [ ]:
df[['r1','r2']].sum()

In [ ]:
df[['r1','r2']].sum(axis=1)

Most methods that use aggregation in their computation have an **axis=** argument. This is used to indicate we want the aggregation column-wise, over axis=0, or row-wise, over axis=1. Note if you do not supply the argument, pandas will use the default axis=0:

So to compute the column-wise Z-score:

In [ ]:
np.sum(df[['r1','r2']],axis=1)

In [ ]:
## compare to:  (df - np.mean(df)) / np.std(df)
(df - df.mean()) / df.std()

Or, for the row-wise Z-score for the first two columns:<br>
Note that the '-' operator does not play well with the result obtained using axis=1  (when we use: dataframe - series).<br>
Basically, the operators will want to do stuff row wise: here, for each row subtract row mean from row ...

In [ ]:
(df - df.mean(axis=1)) / df.std(axis=1)

In [ ]:
## once you get the hang of it, you'll find that pandas behaves very much like Numpy and you can use all the tricks
## for instance: [:,np.newaxis] makes the array explicitly a column vector repeated as many times as nescessary in the context ...
df - df.sum(axis=1)[:,np.newaxis]

In [ ]:
## or use methods with explicitly telling which axis to take
df[['r1','r2']].subtract(df[['r1','r2']].sum(axis=1), axis=0)

There are on the other hand a whole host of extra methods defined on pandas Series (and DataFrame's).
<br><br>
Below we give a few examples, only touching a small subset of capabilities. But, as before, once you understand the basics, you can figure out how to solve specific problems as and when they appear (with the help of google, stackoverflow, ...).

## Predefined Methods

In [ ]:
df = pd.DataFrame(np.random.randint(1,4,size=(10,5)), columns=['c1','c2','c3','c4','c5'])

In [ ]:
## row median
df.median(axis=1)

In [ ]:
## quantiles for each column
df.quantile(q=[0.25])

In [ ]:
## Series methos: the frequency of unique values
df.c1.value_counts(ascending=True)

In [ ]:
## Series method: timeseries specific stats
df.c1.autocorr(lag=1)

In [ ]:
## auto-correlation is the correlation between a series and itself shifted by lag
## as in:
df.c1.corr(df.c1.shift(periods=-1))

In [ ]:
## use map to recode from one set of values to another
mf = pd.Series(['Male','Female','Female','Male','Female'])
pd.concat([mf,mf.map({'Female':1, 'Male':2})], axis=1)

Instead of a dict containing the mappings, the *map()* method can also be called with a function:

In [ ]:
mf.map(lambda x: 1 if (x=='Female') else 2)

This makes *map()* rather powerfull and a good bridge to the next section. In case a pre-canned method does not exist, it is straightforward to implement the functionality yourself using *apply* & *agg*.

## DIY: applymap, apply

When a computation is not supplied out of the box, you can implement it efficiently in pandas using: applymap, apply, & agg.

To apply an arbitrary function to each element in a Series use applymap:

In [ ]:
df = pd.DataFrame([[1,2,3],[4,2,6],[2,2,1]], columns=('c1','c2','c3'))
df

In [ ]:
df.applymap(lambda x: 2*x)

In [ ]:
## apply with axis=0 --> calls the function once for each column 
df.apply(lambda x: x.name) 
## same as: df.apply(lambda x: x.name, axis=0)

In [ ]:
## apply with axis=1 --> calls the function once for each row
df.apply(lambda x: x.name, axis=1)

In [ ]:
## calls the function once for each row --> for each row randomly take three elements
df.apply(lambda x: x.mean(), axis=1)

Note that when the function returns a Series, and the Series get concatenated into a DataFrame, the result will line up the rows based on the column index. You could add a **.reset_index(drop=True)** to make sure all the row sample Series have index=[0,1,2]:

In [ ]:
df.apply( lambda x: x.sample(n=3).reset_index(drop=True), axis=1 )

In [ ]:
df

In [ ]:
df.apply(np.quantile, q=[0,1]).rename(index={0:'MIN',1:'MAX'})

## Windowing: rolling()

Instead of applying a function to each element or each column or each row, you can apply a function to a rolling window of data:

In [ ]:
df = pd.DataFrame(np.random.randn(5,2),columns=('c1','c2'))
df

In [ ]:
## if you want to apply the sum over a window of three elements where two elements exist
df.rolling(3,min_periods=1).median()

In [ ]:
dti = pd.date_range(start='2019-01-01 12:00:00', periods=50, freq='4H')
srs = pd.Series(20*(np.random.rand(50)-0.5).cumsum(), index=dti)
## smooth by taking a rolling mean over 12 hr
srs_smoothed = srs.rolling('12H').mean();
## put both series in a DataFrame
df = pd.DataFrame({'raw':srs,'smoothed':srs_smoothed})
## and plot
df.plot(figsize=(12,5))

## Windowing using an arbitrary function

Just like **apply()** on the whole Series / DataFrame, you can use apply() on the rolling window.<br>
Lets say you are not happy with taking the mean, but you want to weigh based on time delta ...

In [ ]:
df['smooth2'] = srs.rolling(5)\
                   .apply(  lambda vec: np.dot(vec,np.array([1,2,3,4,5])/15), raw=True  ) ## raw = True means the function will receive an ndarray
df.plot(figsize=(12,5))

And as before, you can also use **agg()**. When calling agg() on a Series, you can pass in a dict with multiple functions:

In [ ]:
## same thing, but now using the more generic agg() method (which will work on any function)
df[:10].raw\
       .rolling(3, min_periods=1)\
       .agg({'median': lambda vec: np.median(vec), 
             'range':  lambda vec: max(vec)-min(vec)
            })

I hope you get the flexibility of pandas now. It might not be suited as a big-data tool out of the box, and it does have its downside (my biggest annoyance is not having null's for integers). But, once you get fluent in manipulating your data with pandas, there's nothing really like it :-)

## Accessors
The last section in this part is about accessors. Accessors are used to fascilitate working with strings, dates, and datetimes.

In [ ]:
df = pd.DataFrame([ ('Florence Nightingale', 1820, 1910),
                    ('Karl Pearson', 1857, 1936),
                    ('Ronald Fisher', 1890, 1962),
                    ('Gertrude Cox', 1900, 1978),
                    ('John Tukey', 1915, 2000),
                    ('Carl Gauss', 1777, 1855)
                  ], columns=('name', 'born', 'deceased'))
df

In [ ]:
df.name.str.split()

In [ ]:
type(df.name.str.split())

So we can just string these together ...

In [ ]:
df.name.str.split().str[1].str.upper()

In [ ]:
dti = pd.date_range(start='2019-01-01 12:00:00', periods=5, freq='4H')
df = pd.DataFrame({'dti':dti, 'val':np.random.randn(5)})
df

In [ ]:
df.dti.dt.strftime('%Y%m')

# Changeing Shape

There is a good paper by [Hadley Wickham](https://vita.had.co.nz/papers/tidy-data.pdf) on so called 'tidy' data. In practice, data comes in all shapes and sizes, and cleaning, massaging and prepping data into a shape that is usefull for analysis and visualization often takes up most of the time in Data Science workflows.
<br><br>
In tidy data:
1. Each variable forms a column.
2. Each observation forms a row.
3. Each type of observational unit forms a table.

## Toy Data

In [ ]:
df = pd.read_csv('data/countrystats.csv')
df.head()

In the countrystats data it looks like the  'Indicator Name' column contains many variables. From a 'tidy'
data point of view the different indicators should all have their own column ...

In [ ]:
df['Indicator Name'].value_counts()

### Let's Do Some Cleaning

Clean up the names, so they can be used as column names.<br><br>
To do string manipulation in pandas, we use the string accessor **.str**.<br>
Here it is slightly more involved, looks like we want to define a seperate function to do the string manipulation and then call **apply()**.<br><br>
The function needs to:
1. convert to lower case
2. split the string on a character other than space or [a-z] and take the first
3. strip the whitespace
4. replace remaining spaces for underscores

In [ ]:
## define the function to be applied to all the strings
## basically: use a regular expression to:
## 1. split the substrings made up of charecters not in a-z, A-Z, 0-9 --> re.split('[^a-z0-9]+', s.lower())
## 2. and join the bits with '_' --> '_'.join([...])
def clean_indicator_name(s): return '_'.join([e for e in re.split('[^a-z0-9]+', s.lower()) if e != ''])

In [ ]:
df['Indicator Name'].apply(clean_indicator_name).unique()

In [ ]:
df['Indicator Name'] = df['Indicator Name'].apply(clean_indicator_name)

In [ ]:
## rename the columns
df.rename(columns={'Country Name':   'location',
                   'Indicator Name': 'indicator',
                   'Year':           'year',
                   'Value':          'value'
                  },
          inplace=True
         )

In [ ]:
df.head()

For the purpose of showing the functionality, lets prune the data.<br>
Only keep records where location in ['Argentina','Sweden','United Kingdom'] & year > 1990 and indicator is tax or gdp related:

In [ ]:
df = df[  (df.location.isin(['Argentina','Sweden','United Kingdom'])) & 
          (df.year > 1990) &
          [('tax' in idx) | ('inflation' in idx) for idx in df.indicator]
       ]

In [ ]:
df.head(n=10)

## unstack(): from row index to column index

In [ ]:
df.set_index(['location','indicator','year'], drop=True).head()

Use unstack to 'unstack' a level of the row index by moving it to the column index.<br>
Sound complicated, it is not:
1. create a more meaningfull index for the DataFrame: location, indicator, year
2. unstack the indicator

In [ ]:
## unstack: by default moves the inner-most row index (level=-1) into the column index (inner-most level of the resulting column multi-index)
df.set_index(['location','indicator','year'], drop=True)['value'].unstack()

In [ ]:
df_unstacked = df.set_index(['year','location','indicator'], drop=True)['value'].unstack(level=['year'])
df_unstacked

In [ ]:
df_unstacked.plot.barh(figsize=(20,5), width=0.9);

In [ ]:
## sometimes you want to get rid of the multi-indices, for instance: 
df_unstack = df.set_index(['location','indicator','year'], drop=True).unstack(level='indicator')
df_unstack.head()

In [ ]:
df_unstack.columns

In [ ]:
df_unstack.droplevel(0, axis=1).columns ## axis=1 --> refers to columns (axis=0 --> refers to rows)

In [ ]:
df_unstack.droplevel(0,axis=1).reset_index()

In [ ]:
ax = df.set_index(['indicator','location','year'], drop=True).unstack(level=['year']).plot(kind='bar', figsize=(15,5))
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5));

## stack(): from column index to row index

As expected, stack does the exact oposite:

In [ ]:
df_unstack.stack().head()

In [ ]:
df_unstack.stack().reset_index()

## melt()

In [ ]:
my_df = df.set_index(['location','indicator','year']).unstack(['indicator','year'])
my_df.head()

In [ ]:
my_df.columns = my_df.columns.map(lambda v: v[1] + '_' + str(v[2]))
my_df.head()

In [ ]:
my_df.reset_index(inplace=True)
my_df.head()

In [ ]:
my_df.columns[1:]

In [ ]:
my_df_melted = my_df.melt(id_vars='location', value_vars=my_df.columns[1:])
my_df_melted.head()

## pivot_table()

In [ ]:
my_df_melted.head()

In [ ]:
my_df_melted['indicator'] = my_df_melted.variable.apply(lambda s: '_'.join(s.split('_')[:-1]))
my_df_melted['year']      = my_df_melted.variable.apply(lambda s: s.split('_')[-1])
del my_df_melted['variable']
my_df_melted.head()

In [ ]:
my_df_melted.pivot_table(index=['location','year'], columns='indicator', values='value').reset_index().head()

# Merging

Merging and Joining DataFrames can be done by: 
* concat: concatenate row's or column's depending on axis= --> only support 'inner' joins and 'outer' joins (default)
* join & merge: like the sql join, both can be used to do all of the joins you'd want do

In [ ]:
## lest create some data
lft = pd.DataFrame({'lgrp':[1,1,1,2,2], 'lval': np.random.randint(1,11,size=5)}, index=[2,3,4,5,6])
rgt = pd.DataFrame({'rval': np.random.randint(1,11,size=5)}, index=[3,4,5,6,7])

## concat

In [ ]:
pd.concat([lft,rgt], axis=1)

In [ ]:
pd.concat([lft,rgt], join='inner', axis=1)

Note that the index in not changed, which for column-wise concatenation is natural, but for row-wise concatenation you might want to use *.reset_index()*

In [ ]:
pd.concat([lft,rgt],axis=0,sort=True)

## Merge

In [ ]:
## lest create some data
lft = pd.DataFrame({'key':[2,3,4,5], 'val': np.random.randint(1,11,size=4)})
rgt = pd.DataFrame({'key':[3,4,5,6], 'val': np.random.randint(1,11,size=4)})

In [ ]:
## to do an inner join --> equivalent to how='inner'
pd.merge(left=lft, right=rgt, on='key', suffixes=('_left','_right'))

In [ ]:
pd.merge(left=lft, right=rgt, on='key', how='outer', suffixes=('_left','_right'))

In [ ]:
pd.merge(left=lft, right=rgt, on='key', how='left', suffixes=('_left','_right'))

In [ ]:
pd.merge(left=lft, right=rgt, on='key', how='right', suffixes=('_left','_right'))

# Split-Apply-Combine

One workflow that pops up everywhere, is split-apply-combine: split the dataset row-wise into groups, do some processing on each group individually and then combine the results.<br>
Pandas supports this through **groupby()** which returns a DataFrameGroupBy. Further processing on this DataFrameGroupBy object will apply on each group and combine.

In [ ]:
df = pd.read_csv('data/drinksbycountry.csv')

In [ ]:
df.head()

Also the DataFrameGroupBy is iterable, so:
```python
for grp, df_grp in df.groupby('some_grouping_var'):
    ## do some stuff
```

In [ ]:
dfg = df.groupby('continent')
print(type(dfg))
for grp, df_grp in dfg:
    print(grp, df_grp.shape)

## groupby()

In [ ]:
df.groupby('continent').mean()

In [ ]:
df.groupby('continent')[['beer_servings','spirit_servings','wine_servings']].mean()

### DataFrameGroupBy.apply()

In [ ]:
def process_group_df(df):
    df_pct = df.sum() / df.sum().sum()
    return(df_pct)

In [ ]:
df.groupby('continent')[['beer_servings','spirit_servings','wine_servings']].apply(process_group_df)

In [ ]:
df.groupby('continent')[['beer_servings','spirit_servings','wine_servings']].apply(lambda df: df.sum() / df.sum().sum())

### DataFrameGroupBy.agg()

With the **agg()** you can apply processing on individual columns.

In [ ]:
def process_col(srs):
    return srs.median()
## 
df.groupby('continent')\
  .agg({ 'beer_servings': ['mean', 'sum'],
         'spirit_servings': ['std', 'count'],
         'wine_servings': [process_col]
       })

### DataFrameGroupBy.transform()

With the **transform()** you can apply processing but the DataFrame/Series will have same axis length.<br>
Probably easiest to compare with SQL windowed function with a partition by in the *groupby()*

In [ ]:
df.groupby('continent')[['beer_servings','spirit_servings']].transform('mean').head()

This returns a Series / DataFrame with the same index as the original, so we can concat, merge, join, ... 

In [ ]:
df[['beer_servings','spirit_servings']].head()

So, to create a DateFrame with the continent mean subtracted for beer_servings & spirit_servings:

In [ ]:
(df[['beer_servings','spirit_servings']] - df.groupby('continent')[['beer_servings','spirit_servings']].transform('mean')).head()

# Windowing

Finally, instead of splitting the dataset into distinct groups and process the groups individually, pandas also makes it easy to do processing on a sliding window using **window()**.<br>
This works similar to groupby(), but instead of a DataFrameGroupBy it returns a object of type window.Rolling:

In [ ]:
srs = pd.Series(np.random.randn(100).cumsum(), index=pd.date_range(start='31-01-2019', freq='D', periods=100))
type(srs[:12].rolling(3))

To operate on the pandas.core.window.Rolling, you can call methods directly, of operate use apply and operate on the numpy array that is passed in each time ...

In [ ]:
srs[:12].rolling(3).median()

Note: currently a numpy array is passed in. In future (probably from 1.0 onward) a Series will be passed in.<br>
(The raw=True is used to silence the warning message regarding this changing behaviour)

In [ ]:
srs[:12].rolling(3).apply(lambda win: np.median(win), raw=True)

# Basic Visualizing

Jupyter / pandas has some usefull facitilies to highlight elements in the output.<br>
It also has some plot methods directly defined on the Series / DataFrame.<br>

In [ ]:
## lets create a DataFrame with the same row & column index
df = pd.DataFrame(np.random.randn(4,3), index=[1,2,3,4], columns=[1,2,3])
df

In [ ]:
## return some css styling based on the value
def vis_highlight_negative(cell):
    return(f"{'color: blue' if (cell < -1) else 'color: black'}; " +
           f"{'background: yellow' if (cell < -1) else ''}"
          )

In [ ]:
## applymap: function gets called for each cell in the DataFrame
df.style.applymap(vis_highlight_negative)

In [ ]:
def vis_highlight_column_max(srs):
    cmax = srs.max()
    cmin = srs.min()
    bg   = {0: '', 1: 'background: red', 2: 'background: green'}
    return([bg[1*(cell==cmin) + 2*(cell==cmax)] for cell in srs])

In [ ]:
df.style.apply(vis_highlight_column_max)

Timeseries plots

In [ ]:
srs = pd.Series(np.random.randn(100).cumsum(), index=pd.date_range(start='31-01-2019', freq='D', periods=100))

In [ ]:
srs.plot();

In [ ]:
srs.plot(figsize=(15,4), color='red', grid=(True,True));

Another powerfull feature of pandas is its **rolling()** functionality. This works much like **groupby** but allows us to operate on a rolling window.<br>
For instance to cumpute a rolling median on a window of size 3:

In [ ]:
srs.rolling(3).median().plot()

In [ ]:
pd.concat([srs,srs.rolling(5).median()],axis=1).rename(columns={0:'raw',1:'median smoothed'}).plot(figsize=(20,4));

# Read / Write External Data

I should have probably started with this! But Pandas has a powerfull set of functions top read and write data in a multitude of formats.<br>
There is a lot to say about this, here I just want to pull out a couple of things:
* usually data is obtained from a DB using **read_sql**
* you can read sas data using **read_sas**
* a very efficient / fast way to store Python objects is through **read_pickle** (or df.to_pickle to store)
* a format that supports fast and efficient sharing between Python and R is **feather** (need to install pyarrow!)

In [ ]:
[m for m in dir(pd) if m.startswith('read_')]

## CSV

In [ ]:
with open('data/drinksbycountry.csv','r') as f: lines = f.readlines(300)
lines

In [ ]:
%%time
df = pd.read_csv('data/drinksbycountry.csv')

In [ ]:
df.head()

## Pickle

In [ ]:
df.to_pickle('data/drinksbycountry.pickle')

In [ ]:
df = pd.read_pickle('data/drinksbycountry.pickle')

In [ ]:
df.head()

In [ ]:
%%time
df_bigger = pd.read_csv('data/countrystats.csv')

In [ ]:
df_bigger.head()

In [ ]:
df_bigger.to_pickle('data/countrystats.pickle')

In [ ]:
%%time
df_bigger = pd.read_pickle('data/countrystats.pickle')

The performance gains are much bigger for larger datasets!! This toy example does not do it justice.

## Feather: cannot use at the moment!!

In [ ]:
import pyarrow

In [ ]:
pyarrow.feather.write_feather(df_bigger, 'data/countrystats.feather')

In [ ]:
df_bigger = pyarrow.feather.read_feather('data/countrystats.feather', columns=None, use_threads=True)

# Dask DataFrame